## Module 4 - In this jupyter notebook, land and water productivity are calculated 
* Step 4a - Set up
* Step 4b - Calculate land productivity: i) biomass and ii) crop yield  
* Step 4c - Calculate productivity: i) biomass water productivity and ii) crop water productivity
**=====================================================================================================================**

![title](img/Fig4_1.png)

**=====================================================================================================================**

## Step 4a - Setu up

## i) Import packages/libraries

In [1]:
import os
import glob

import numpy as np
from matplotlib import pyplot as plt
master_dr = os.path.split(os.getcwd())[0]

# change the directory to where the modules are saved
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## Step 4b - Calculate land productivity
* i) Biomass
* ii) Crop yield

## i) Calculate biomass
$Biomass   = AOT*f_c*\frac{NPP*22.222}{1-mc}$
<br/>where AOT is the above ground over total biomass production ration (-), fc is the light use efficiency correction factor[-]. NPP is seasonal net primary production in gC/m²/season, mc is moisture content in the fresh biomass [-] and HI is harvest index

## * Import raster data

In [2]:
# get seasonal data
seasonal_dr = os.path.join(master_dr, r"Data/tif/seasonal")
monthly_dr = os.path.join(master_dr, r"Data/tif/monthly")
T_seasonal = os.path.join(seasonal_dr, f"seasonal_T.tif")
AETI_seasonal = os.path.join(seasonal_dr, f"seasonal_AETI.tif")
ETp_seasonal = os.path.join(seasonal_dr, f"seasonal_RET.tif")
NPP_seasonal = os.path.join(seasonal_dr, f"seasonal_NPP.tif")
AETI_monthly = sorted(glob.glob(f"{monthly_dr}/AETI"+"/*.tif"))
NPP_monthly = sorted(glob.glob(f"{monthly_dr}/NPP"+"/*.tif"))

## ** Crop parameters
![title](img/Fig4_2.png)

In [3]:
# Update the crop parameters specic to the crop
MC = 0.15  # moisture content, dry matter over freshbiomass
fc = 1  # Light use efficiency correction factor
AOT= 0.85  # above ground over total biomass production ratio(AOT)

## **** Calculate above ground biomass (AGBM)

In [4]:
# collecting Geoinfo such as projection, the x and y axis
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(AETI_seasonal)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

NPP  = gis.OpenAsArray(NPP_seasonal, nan_values=True)
AGBM = (AOT * fc * (NPP * 22.222 / (1 - MC))) / 1000  # Above ground biomass, 1000 is to covert from kg to ton

# save into output folder
biomass_seasonal    = os.path.join(seasonal_dr, f"seasonal_biomass.tif")
gis.CreateGeoTiff(biomass_seasonal, AGBM, driver, NDV, xsize, ysize, GeoT, Projection)

# monthly biomass
biomass_monthly = os.path.join(monthly_dr,"biomass")
os.makedirs(biomass_monthly, exist_ok= True)
for NPP_month in NPP_monthly:
  NPP  = gis.OpenAsArray(NPP_month, nan_values=True)
  AGBM = (AOT * fc * (NPP * 22.222 / (1 - MC))) / 1000  # Above ground biomass, 1000 is to covert from kg to ton
  
  # save into output folder
  biomass_month    = os.path.join(biomass_monthly, os.path.basename(NPP_month).replace("NPP", "biomass"))
  gis.CreateGeoTiff(biomass_month, AGBM, driver, NDV, xsize, ysize, GeoT, Projection)

## ii) Calculate crop yield
* $Crop yield = HI*B$
* where HI is harvest index, and B (or AGBM) is above ground biomass

## ** Update the Harvest index

In [5]:
HI = 0.45     # harvest index of the crop

## **** Calculate crop yield

In [6]:
# collecting Geoinfo such as projection, the x and y axis
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(AETI_seasonal)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster


AGBM      = gis.OpenAsArray(biomass_seasonal, nan_values=True)
CropYield = HI*AGBM
    
# save into output folder
yield_seasonal    = os.path.join(seasonal_dr, f"seasonal_yield.tif")
gis.CreateGeoTiff(yield_seasonal, CropYield, driver, NDV, xsize, ysize, GeoT, Projection) 

# monthly yield
biomass_monthly = sorted(glob.glob(f"{monthly_dr}/biomass"+"/*.tif"))
yield_monthly = os.path.join(monthly_dr,"yield")
os.makedirs(yield_monthly, exist_ok= True)
for biomass_month in biomass_monthly:
  AGBM      = gis.OpenAsArray(biomass_month, nan_values=True)
  CropYield = HI*AGBM
  
  # save into output folder
  yield_month    = os.path.join(yield_monthly, os.path.basename(biomass_month).replace("biomass", "yield"))
  gis.CreateGeoTiff(yield_month, CropYield, driver, NDV, xsize, ysize, GeoT, Projection)

## Step 4c - Calculate water productivity (WP)

## i) Calculate biomass water productivity 
$Biomass WP= \frac{Biomass}{ET_a}$

## **** Calculate biomass water productivity (WPb)

In [7]:
# collecting Geoinfo such as projection, the x and y axis
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(AETI_seasonal)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

## Calculate the WP
AETI = gis.OpenAsArray(AETI_seasonal, nan_values=True) 
AGBM = gis.OpenAsArray(biomass_seasonal, nan_values=True)
WPb  = AGBM/AETI*100

# save into output folder
wpb_seasonal    = os.path.join(seasonal_dr, f"seasonal_WPb.tif")
gis.CreateGeoTiff(wpb_seasonal, WPb, driver, NDV, xsize, ysize, GeoT, Projection)

# monthly WPb
biomass_monthly = sorted(glob.glob(f"{monthly_dr}/biomass"+"/*.tif"))
AETI_monthly = sorted(glob.glob(f"{monthly_dr}/AETI"+"/*.tif"))
WPb_monthly = os.path.join(monthly_dr,"WPb")
os.makedirs(WPb_monthly, exist_ok= True)
for AETI_month, biomass_month in zip(AETI_monthly, biomass_monthly):
  AETI = gis.OpenAsArray(AETI_month, nan_values=True)
  AGBM = gis.OpenAsArray(biomass_month, nan_values=True)
  WPb  = AGBM/AETI*100
  # save into output folder
  WPb_month    = os.path.join(WPb_monthly, os.path.basename(biomass_month).replace("biomass", "WPb"))
  gis.CreateGeoTiff(WPb_month, WPb, driver, NDV, xsize, ysize, GeoT, Projection)

## ii) Calculate crop water productivity 
$Crop WP = \frac{CropYield}{ET_a}$

## **** Calculate crop water productivity (WPy)

In [8]:
# collecting Geoinfo such as projection, the x and y axis
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(AETI_seasonal)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

## Calculate the WP
AETI  = gis.OpenAsArray(AETI_seasonal, nan_values=True) 
Yield = gis.OpenAsArray(yield_seasonal, nan_values=True)
WPy   = Yield/AETI*100

wpy_seasonal    = os.path.join(seasonal_dr, "seasonal_WPy.tif")
gis.CreateGeoTiff(wpy_seasonal, WPy, driver, NDV, xsize, ysize, GeoT, Projection)

# monthly WPy
yield_monthly = sorted(glob.glob(f"{monthly_dr}/yield"+"/*.tif"))
AETI_monthly = sorted(glob.glob(f"{monthly_dr}/AETI"+"/*.tif"))
WPy_monthly = os.path.join(monthly_dr,"WPy")
os.makedirs(WPy_monthly, exist_ok= True)
for AETI_month, yield_month in zip(AETI_monthly, yield_monthly):
  AETI = gis.OpenAsArray(AETI_month, nan_values=True)
  Yield = gis.OpenAsArray(yield_month, nan_values=True)
  WPy  = Yield/AETI*100
  # save into output folder
  WPy_month    = os.path.join(WPy_monthly, os.path.basename(yield_month).replace("yield", "WPy"))
  gis.CreateGeoTiff(WPy_month, WPy, driver, NDV, xsize, ysize, GeoT, Projection)